# Homepage Demo
This notebook was created to support the basic example that is showcased on the VeRoViz homepage (https://veroviz.org). 

The goal is to demonstrate a common workflow (create nodes, calculate distances, solve problem, visualize solutions) via a simple example.

---
## Import VeRoViz

We first need to import the VeRoViz Python package:

In [1]:
import veroviz as vrv
import os

vrv.checkVersion()

'Your current installed version of veroviz is 0.2.1. You are up-to-date with the latest available version.'

## Select a Data Provider

Now, we'll specify a data provider, and any additional arguments required by the chosen data provider.

On the VeRoViz homepage, we used 'OSRM'.  Here, we'll use 'ORS'. 

- See https://veroviz.org/docs/dataproviders.html for other `dataProvider` options.
- See https://veroviz.org/documentation.html for information about setting environment variables (e.g., `os.environ['ORSKEY']`).

In [2]:
# DATA_PROVIDER = 'OSRM-online'
# DATA_PROVIDER_ARGS = {}

DATA_PROVIDER = 'ORS-online'
DATA_PROVIDER_ARGS = {'APIkey': os.environ['ORSKEY'], 'databaseName': None}

## Create a Bounding Region

We're going to automatically generate some random nodes on the map.  To do this, we'll need to define a bounding region, within which the nodes will be created.

The [Sketch](https://veroviz.org/sketch.html) tool makes it easy to create a bounding region.  The [lat, lon] points below were copied from Sketch.

In [3]:
myBoundary = [[42.914949262700084, -78.90020370483398], 
              [42.871938424448466, -78.89556884765626], 
              [42.875083507773496, -78.82158279418947], 
              [42.91293772859624, -78.81729125976564]]

In [4]:
# Create a map that shows just the boundary:
vrv.createLeaflet(boundingRegion = myBoundary)

## Generate Nodes within the Boundary

We'll generate two types of nodes:
1. A single depot (shown with a red "home" icon), and
2. Four customer nodes (with blue "star" icons).

These nodes will be uniformly distributed over the boundary region.

- See https://veroviz.org/docs/veroviz.generateNodes.html for more information on the `generateNodes()` options.

In [5]:
# Single depot node:
nodesDF = vrv.generateNodes(
    numNodes         = 1,
    startNode        = 0,
    nodeType         = 'depot',
    leafletColor     = 'red',
    leafletIconType  = 'home',
    nodeDistrib      = 'uniformBB', 
    nodeDistribArgs  = { 'boundingRegion': myBoundary },
    dataProvider     = DATA_PROVIDER,
    dataProviderArgs = DATA_PROVIDER_ARGS,
    snapToRoad       = True)

In [6]:
# 4 Customer nodes:
nodesDF = vrv.generateNodes(
    initNodes       = nodesDF,
    numNodes        = 4,
    startNode       = 1,
    nodeType        = 'customer',
    leafletColor    = 'blue',
    leafletIconType = 'star',
    nodeDistrib     = 'uniformBB', 
    nodeDistribArgs = { 'boundingRegion': myBoundary },
    dataProvider     = DATA_PROVIDER,
    dataProviderArgs = DATA_PROVIDER_ARGS,
    snapToRoad      = True)

We can now inspect the "nodes" dataframe that was generated.

- Details on the columns in the "nodes" dataframe (as well as information on the other pandas dataframe structures used by VeRoViz) may be found at https://veroviz.org/docs/dataframes.html.

In [7]:
# Display the nodesDF dataframe:
nodesDF

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,0,42.902761,-78.850217,0,None,depot,glyphicon,home,red,0,pin,Cesium.Color.BLUE,0
1,1,42.876137,-78.833208,0,None,customer,glyphicon,star,blue,1,pin,Cesium.Color.BLUE,1
2,2,42.881240,-78.832692,0,None,customer,glyphicon,star,blue,2,pin,Cesium.Color.BLUE,2
3,3,42.876817,-78.862850,0,None,customer,glyphicon,star,blue,3,pin,Cesium.Color.BLUE,3
4,4,42.907439,-78.875042,0,None,customer,glyphicon,star,blue,4,pin,Cesium.Color.BLUE,4


The `createLeaflet()` function can be used to display the boundary region and our nodes.

- See https://veroviz.org/docs/veroviz.createLeaflet.html for more options.

In [8]:
# Create a map that shows the boundary and the nodes:
vrv.createLeaflet(nodes          = nodesDF, 
                  boundingRegion = myBoundary,
                  mapBackground  = 'Arcgis Roadmap')

## Create Travel Matrices

- See https://veroviz.org/docs/veroviz.getTimeDist2D.html for more options.
- There is also a 3D function (https://veroviz.org/docs/veroviz.getTimeDist3D.html), which is suitable for flying vehicles like drones.


In [9]:
[time, dist] = vrv.getTimeDist2D(nodes            = nodesDF, 
                                 matrixType       = 'all2all',
                                 routeType        = 'fastest', 
                                 dataProvider     = DATA_PROVIDER,
                                 dataProviderArgs = DATA_PROVIDER_ARGS)

In [10]:
print("'time' Dictionary (units in seconds): ")
print(time)

print("\nTime from depot (node 0) to customer 3 (node 3): ", time[0,3], "seconds")

print("\n'dist' Dictionary (units in meters):")
print(dist)

print("\nDistance from depot to customer 3: ", dist[0,3], "meters")

'time' Dictionary (units in seconds): 
{(0, 0): 0.0, (0, 1): 420.32, (0, 2): 344.0, (0, 3): 395.9, (0, 4): 201.91, (1, 0): 415.78, (1, 1): 0.0, (1, 2): 137.41, (1, 3): 307.85, (1, 4): 581.24, (2, 0): 339.46, (2, 1): 137.41, (2, 2): 0.0, (2, 3): 285.71, (2, 4): 512.19, (3, 0): 395.28, (3, 1): 307.85, (3, 2): 285.71, (3, 3): 0.0, (3, 4): 417.99, (4, 0): 201.91, (4, 1): 581.24, (4, 2): 512.19, (4, 3): 417.99, (4, 4): 0.0}

Time from depot (node 0) to customer 3 (node 3):  395.9 seconds

'dist' Dictionary (units in meters):
{(0, 0): 0.0, (0, 1): 4337.07, (0, 2): 3803.5, (0, 3): 4505.21, (0, 4): 2467.01, (1, 0): 4280.3, (1, 1): 0.0, (1, 2): 969.83, (1, 3): 3114.59, (1, 4): 6506.16, (2, 0): 3746.73, (2, 1): 969.83, (2, 2): 0.0, (2, 3): 3126.69, (2, 4): 6203.56, (3, 0): 4497.41, (3, 1): 3114.59, (3, 2): 3126.69, (3, 3): 0.0, (3, 4): 4901.49, (4, 0): 2467.01, (4, 1): 6506.16, (4, 2): 6203.56, (4, 3): 4901.49, (4, 4): 0.0}

Distance from depot to customer 3:  4505.21 meters


## Generate a dummy solution

This is where the user would typically apply their own algorithm to solve a problem.

For demonstration purposes, let's create a solution "by hand".  The solution should have the following features:
1. There should be two (2) vehicles:
   - One truck, and 
   - One UAV/drone
2. Both vehicles should start/end at the depot node (node 0).
   - The truck should go from the depot to 1 to 2 and back to the depot.
   - The UAV should go from the depot to 3 with a package; from 3 back to the depot empty; from the depot to 4 with a package; and return to the depot empty.
3. Each vehicle should spend 30 seconds at each node.
4. Each vehicle should drop a package at each customer.
5. The truck should follow the road network.
6. The UAV will have a "square" flight path, meaning that it will takeoff and land vertically, and fly horizontally at a constant altitude.  

The `dummySolver()` function will return an "assignments" dataframe.  
- See https://veroviz.org/docs/assignments.html for details on the "assignments" dataframe structure.

In [11]:
def dummySolver(nodesDF, dist, time):
    import pandas as pd
    
    # Assume truck travels depot -> 1 -> 2 -> depot
    route = {'truck': [0, 1, 2, 0], 
             'drone': [0, 3, 0, 4, 0]}
  
    configs = {'truck': {
                    'vehicleModels': ['veroviz/models/ub_truck.gltf'],
                    'leafletColor': 'blue',
                    'cesiumColor': 'Cesium.Color.BLUE',
                    'packageModel': 'veroviz/models/box_blue.gltf',
                    'modelScale': 100,
                    'minPxSize': 45 }, 
               'drone': {'vehicleModels': ['veroviz/models/drone.gltf', 'veroviz/models/drone_package.gltf'],
                    'leafletColor': 'orange',
                    'cesiumColor': 'Cesium.Color.ORANGE',
                    'packageModel': 'veroviz/models/box_yellow.gltf',
                    'modelScale': 100,
                    'minPxSize': 45 }
              }
        
    serviceTime = 30 # seconds    
    
    # Initialize an empty "assignments" dataframe.  
    assignmentsDF = vrv.initDataframe('assignments')

    for vehicle in route:
        startTime = 0
        for i in list(range(0, len(route[vehicle])-1)):
            startNode = route[vehicle][i]
            endNode   = route[vehicle][i+1]

            startLat  = nodesDF[nodesDF['id'] == startNode]['lat'].values[0]
            startLon  = nodesDF[nodesDF['id'] == startNode]['lon'].values[0]
            endLat    = nodesDF[nodesDF['id'] == endNode]['lat'].values[0]
            endLon    = nodesDF[nodesDF['id'] == endNode]['lon'].values[0]

            if ((vehicle == 'drone') and (startNode == 0)):
                # Use the 3D model of a drone carrying a package
                myModel = configs[vehicle]['vehicleModels'][1]
            else:
                # Use the 3D model of either a delivery truck or an empty drone
                myModel = configs[vehicle]['vehicleModels'][0]

            if (vehicle == 'truck'):
                # Get turn-by-turn navigation for the truck, as it travels
                # from the startNode to the endNode:
                shapepointsDF = vrv.getShapepoints2D(
                    # odID           = odID,
                    objectID         = vehicle, 
                    modelFile        = myModel,
                    modelScale       = configs[vehicle]['modelScale'], 
                    modelMinPxSize   = configs[vehicle]['minPxSize'], 
                    startTimeSec     = startTime,
                    startLoc         = [startLat, startLon],
                    endLoc           = [endLat, endLon],
                    # expDurationSec = time[startNode, endNode], 
                    routeType        = 'fastest',
                    leafletColor     = configs[vehicle]['leafletColor'], 
                    # leafletWeight  = 3, 
                    # leafletStyle   = myArcStyle, 
                    # leafletOpacity = 0.8,
                    cesiumColor      = configs[vehicle]['cesiumColor'], 
                    # cesiumWeight   = 3, 
                    # cesiumStyle    = myArcStyle, 
                    # cesiumOpacity  = 0.8,
                    dataProvider     = DATA_PROVIDER,
                    dataProviderArgs = DATA_PROVIDER_ARGS) 
            else:
                # Get a 3D flight profile for the drone:
                shapepointsDF = vrv.getShapepoints3D(
                    # odID             = odID,
                    objectID           = vehicle, 
                    modelFile          = myModel,
                    modelScale         = configs[vehicle]['modelScale'], 
                    modelMinPxSize     = configs[vehicle]['minPxSize'], 
                    startTimeSec       = startTime,
                    startLoc           = [startLat, startLon],
                    endLoc             = [endLat, endLon],
                    takeoffSpeedMPS    = 5,                # FIXME
                    cruiseSpeedMPS     = 20,               # FIXME
                    landSpeedMPS       = 3,                # FIXME
                    cruiseAltMetersAGL = 100,              # FIXME
                    routeType          = 'square',
                    cesiumColor        = configs[vehicle]['cesiumColor']) 
                    # cesiumWeight     = 3, 
                    # cesiumStyle      = myArcStyle, 
                    # cesiumOpacity    = 0.8)
                
            # Update the assignments dataframe:
            assignmentsDF = pd.concat([assignmentsDF, shapepointsDF], ignore_index=True, sort=False)

            # Update the time
            startTime = max(shapepointsDF['endTimeSec'])

            # Add loitering for service
            assignmentsDF = vrv.addStaticAssignment(
                initAssignments      = assignmentsDF, 
                # odID                 = odID, 
                objectID             = vehicle, 
                modelFile            = myModel, 
                modelScale           = configs[vehicle]['modelScale'], 
                modelMinPxSize       = configs[vehicle]['minPxSize'], 
                loc                  = [endLat, endLon],
                startTimeSec    = startTime,
                endTimeSec = startTime + serviceTime)

            # odID += 1

            # Update the time again
            startTime = startTime + serviceTime

            # Add a package at all non-depot nodes:
            if (endNode != 0):
                assignmentsDF = vrv.addStaticAssignment(
                    initAssignments      = assignmentsDF, 
                    # odID                 = 0, 
                    objectID             = 'package %d' % endNode,
                    modelFile            = configs[vehicle]['packageModel'], 
                    modelScale           = 20, 
                    modelMinPxSize       = 30, 
                    loc                  = [endLat, endLon],
                    startTimeSec    = startTime,
                    endTimeSec = -1)
        
    return assignmentsDF

### Call our function to get a solution:

In [12]:
assignmentsDF = dummySolver(nodesDF, dist, time)
assignmentsDF

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,1,truck,/veroviz/models/ub_truck.gltf,100,45,0,42.902761,-78.850217,0,10.2502,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
1,1,truck,/veroviz/models/ub_truck.gltf,100,45,10.2502,42.901993,-78.850208,0,19.806,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
2,1,truck,/veroviz/models/ub_truck.gltf,100,45,19.806,42.901277,-78.850207,0,34.5,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
3,1,truck,/veroviz/models/ub_truck.gltf,100,45,34.5,42.900176,-78.850204,0,53.2103,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
4,1,truck,/veroviz/models/ub_truck.gltf,100,45,53.2103,42.900165,-78.849014,0,55.7101,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
5,1,truck,/veroviz/models/ub_truck.gltf,100,45,55.7101,42.900164,-78.848855,0,62.1247,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
6,1,truck,/veroviz/models/ub_truck.gltf,100,45,62.1247,42.900162,-78.848447,0,64.2,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
7,1,truck,/veroviz/models/ub_truck.gltf,100,45,64.2,42.900162,-78.848315,0,78.6059,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
8,1,truck,/veroviz/models/ub_truck.gltf,100,45,78.6059,42.898543,-78.848311,0,86.7387,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8
9,1,truck,/veroviz/models/ub_truck.gltf,100,45,86.7387,42.897629,-78.848307,0,107.276,...,0,blue,3,solid,0.8,True,Cesium.Color.BLUE,3,solid,0.8


## Visualize the dummy solution

We'll create a static Leaflet map and a dynamic Cesium "movie".

### Static Leaflet map

In [13]:
# Create a Leaflet map showing the nodes and the routes.
vrv.createLeaflet(nodes=nodesDF, arcs=assignmentsDF)

### Dynamic Cesium movie

In [ ]:
# Create a Cesium movie showing the nodes, routes, and package deliveries.
vrv.createCesium(
    assignments = assignmentsDF, 
    nodes       = nodesDF, 
    startTime   = '10:00:00', 
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'homepage_demo')